In [1]:
#import 
import os
import sys
import gzip
import random
import platform
import warnings
import collections
from tqdm import tqdm, tqdm_notebook
import re
from sklearn.metrics import f1_score
import requests
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.datasets import load_iris
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler


In [2]:
# Random Seed Set
SEED = 17
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)  # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = True  # type: ignore

In [3]:
# 현재 OS 및 라이브러리 버전 체크 체크
current_os = platform.system()
print(f"Current OS: {current_os}")
print(f"CUDA: {torch.cuda.is_available()}")
print(f"Python Version: {platform.python_version()}")
print(f"torch Version: {torch.__version__}")
print(f"torchvision Version: {torchvision.__version__}")

# 중요하지 않은 에러 무시
warnings.filterwarnings(action='ignore')

# 유니코드 깨짐현상 해결
mpl.rcParams['axes.unicode_minus'] = False

Current OS: Linux
CUDA: True
Python Version: 3.8.5
torch Version: 1.7.1
torchvision Version: 0.8.2


In [4]:
test_dir = '/opt/ml/input/data/eval'
train_dir = '/opt/ml/input/data/train'
train_img_dir = train_dir+'/images'
test_img_dir = test_dir + "/images"
data_info = train_dir + '/train.csv'

In [5]:
data = pd.read_csv(train_dir+"/train.csv")
list_path = data['path'].to_list()
condition_list = [
    (data['age']<30),
    (30<=data['age'])&(data['age']<60),
    (data['age']>=60)
]
condition_names = ["lower30","30to60","upper60"]
data['age_cate'] = np.select(condition_list, condition_names)
dict_label = {path : data[data['path']==path][['gender', 'age_cate']].values.tolist()[0] for path in list_path}

In [ ]:
def read_images(path, labels = None, mode= None):
    if labels:
        train_x, train_y = [], []

        if mode == "mask":
            for label in labels:
                label_dir = path+"/"+label
                file_list = [x for x in os.listdir(label_dir) if x[0] != "."]
                for file_name in  file_list:
                    img = Image.open(label_dir+"/"+file_name)
                    train_x.append(img)
                    if "incorrect" in file_name :
                        train_y.append(2)
                    elif "normal" in file_name:
                        train_y.append(1)
                    elif "mask" in file_name:
                        train_y.append(0)
            return train_x, train_y

        elif mode == "age_cate":
            for label in labels:
                label_dir = path+"/"+label
                file_list = [x for x in os.listdir(label_dir) if x[0] != "."]
                for file_name in  file_list:
                    img = Image.open(label_dir+"/"+file_name)
                    train_x.append(img)
                    age_cate = dict_label[label][1]
                    if age_cate == "upper60":
                        train_y.append(2)
                    elif age_cate == "30to60":
                        train_y.append(1)
                    elif age_cate == "lower30":
                        train_y.append(0)
            return train_x, train_y
            
        elif mode == "gender":
            for label in labels:
                label_dir = path+"/"+label
                file_list = [x for x in os.listdir(label_dir) if x[0] != "."]
                for file_name in  file_list:
                    img = Image.open(label_dir+"/"+file_name)
                    train_x.append(img)
                    gender = dict_label[label][0]
                    if gender == "female":
                        train_y.append(1)
                    elif gender == "male":
                        train_y.append(0)
            return train_x, train_y
 
    else:
        train_x = []
        id_list = []
        file_list = [x for x in os.listdir(path) if x[0] != "."]
        for file_name in  file_list:
            id_list.append(file_name)
            img = Image.open(path+"/"+file_name)
            train_x.append(img)
        return train_x, id_list
    

In [8]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=0.5, std=0.5, inplace=True),
                                transforms.Resize((128,128))])

In [9]:
class MASKDataset(Dataset):
    def __init__(self, path, transform=None, train=True, labels=None, mode = None):
        if train :
            self.mode = mode
            self.X, self.y = read_images(path, labels,mode)
        else:
            self.X, self.id_list = read_images(path)
        
        self.train = train
        self._repr_indent = 4
        self.path = path
        self.transform = transform
        self.classes = ['Wear', 'NotWear', 'Incorrect']
 

    def __len__(self):
        return len(self.X)

    def get_id_list(self):
        return self.id_list
        
    def __getitem__(self, idx):
        if self.train :
            X,y = self.X[idx], self.y[idx]
        else:
            X = self.X[idx]
        if self.transform:
            X = self.transform(X)
        if self.train :
            return X, y
        else:
            return X


    def __repr__(self):
        '''
        https://github.com/pytorch/vision/blob/master/torchvision/datasets/vision.py
        '''
        head = "MASK Dataset\n"
        data_path = self._repr_indent*" " + "Data path: {}".format(self.path)
        num_data = self._repr_indent*" " + "Number of datapoints: {}".format(self.__len__())
        num_classes = self._repr_indent*" " + "Number of classes: {}".format(len(self.classes))

        return '\n'.join([head,
                          data_path, 
                          num_data, num_classes])

In [10]:
test_idx = pd.Series(pd.Series(list_path).index).sample(frac=0.2, random_state=SEED).to_list()
test_labels = [list_path[idx] for idx in test_idx]
train_labels = pd.Series(list_path)
train_labels = train_labels.drop(test_idx).to_list()

In [11]:
len(train_labels), len(test_labels)

(2160, 540)

In [94]:
m_train_set = MASKDataset(path=train_img_dir, train=True, labels=train_labels, transform=transform, mode="mask")
m_validation_set = MASKDataset(path=train_img_dir, train=True, labels=test_labels, transform=transform, mode="mask")
m_total_set = MASKDataset(path=train_img_dir, train=True, labels=list_path, transform=transform, mode="mask")

g_train_set = MASKDataset(path=train_img_dir, train=True, labels=train_labels, transform=transform, mode="gender")
g_validation_set = MASKDataset(path=train_img_dir, train=True, labels=test_labels, transform=transform, mode="gender")
g_total_set = MASKDataset(path=train_img_dir, train=True, labels=list_path, transform=transform, mode="gender")

a_train_set = MASKDataset(path=train_img_dir, train=True, labels=train_labels, transform=transform, mode="age_cate")
a_validation_set = MASKDataset(path=train_img_dir, train=True, labels=test_labels, transform=transform, mode="age_cate")
a_total_set = MASKDataset(path=train_img_dir, train=True, labels=list_path, transform=transform, mode="age_cate")


In [13]:
eval_set = MASKDataset(path=test_img_dir, train=False, transform=transform)

In [14]:
len(g_train_set), len(g_validation_set), len(g_total_set), len(eval_set)


(15120, 3780, 18900, 12600)

In [15]:
m_train_loader = DataLoader(dataset=m_train_set, batch_size=16, shuffle=True,num_workers=0)
m_valid_loader = DataLoader(dataset=m_validation_set, batch_size=16, shuffle=True, num_workers=0)
m_total_loader = DataLoader(dataset=m_total_set, batch_size=16, shuffle=True, num_workers=0)

g_train_loader = DataLoader(dataset=g_train_set, batch_size=16, shuffle=True,num_workers=0)
g_valid_loader = DataLoader(dataset=g_validation_set, batch_size=16, shuffle=True, num_workers=0)
g_total_loader = DataLoader(dataset=g_total_set, batch_size=16, shuffle=True, num_workers=0)

a_train_loader = DataLoader(dataset=a_train_set, batch_size=16, shuffle=True,num_workers=0)
a_valid_loader = DataLoader(dataset=a_validation_set, batch_size=16, shuffle=True, num_workers=0)
a_total_loader = DataLoader(dataset=a_total_set, batch_size=16, shuffle=True, num_workers=0)

In [16]:
eval_loader = DataLoader(dataset=eval_set, batch_size=1, shuffle=False, num_workers=0)

In [17]:
import torchvision.models.resnet as resnet
import torch.nn as nn
import torch.optim as optim

conv1x1=resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock= resnet.BasicBlock
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [18]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=3, zero_init_residual=True):
        super().__init__()
        self.inplanes = 32
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 32, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 64, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 128, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 256, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(256* block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight,0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1 ):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion :
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes*block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion)
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x= self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x



In [19]:
resnet50 = ResNet(resnet.Bottleneck, [3, 4, 6, 3], 3, True).to(device)
resnet50_gender = ResNet(resnet.Bottleneck, [3, 4, 6, 3], 2, True).to(device)
# resnet50

In [20]:
from torchsummary import summary
summary(resnet50, input_size=(3,128,128), device=device)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             864
       BatchNorm2d-2         [-1, 32, 128, 128]              64
              ReLU-3         [-1, 32, 128, 128]               0
         MaxPool2d-4           [-1, 32, 64, 64]               0
            Conv2d-5           [-1, 32, 64, 64]           1,024
       BatchNorm2d-6           [-1, 32, 64, 64]              64
              ReLU-7           [-1, 32, 64, 64]               0
            Conv2d-8           [-1, 32, 64, 64]           9,216
       BatchNorm2d-9           [-1, 32, 64, 64]              64
             ReLU-10           [-1, 32, 64, 64]               0
           Conv2d-11          [-1, 128, 64, 64]           4,096
      BatchNorm2d-12          [-1, 128, 64, 64]             256
           Conv2d-13          [-1, 128, 64, 64]           4,096
      BatchNorm2d-14          [-1, 128,

In [21]:
summary(resnet50_gender, input_size=(3,128,128), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             864
       BatchNorm2d-2         [-1, 32, 128, 128]              64
              ReLU-3         [-1, 32, 128, 128]               0
         MaxPool2d-4           [-1, 32, 64, 64]               0
            Conv2d-5           [-1, 32, 64, 64]           1,024
       BatchNorm2d-6           [-1, 32, 64, 64]              64
              ReLU-7           [-1, 32, 64, 64]               0
            Conv2d-8           [-1, 32, 64, 64]           9,216
       BatchNorm2d-9           [-1, 32, 64, 64]              64
             ReLU-10           [-1, 32, 64, 64]               0
           Conv2d-11          [-1, 128, 64, 64]           4,096
      BatchNorm2d-12          [-1, 128, 64, 64]             256
           Conv2d-13          [-1, 128, 64, 64]           4,096
      BatchNorm2d-14          [-1, 128,

In [22]:
class Config:
  def __init__(self, **kwargs):
    for key, value in kwargs.items():
      setattr(self, key, value)

In [23]:
lr = 0.0008
epochs = 30
optimizer = 'Adam'

In [36]:
# 파라미터 클래스
config1 = Config(
    trainloader = m_train_loader,
    testloader = m_valid_loader,
    model = resnet50,
    device = device,
    optimizer = torch.optim.Adam(resnet50.parameters(), lr=lr),
    criterion= nn.CrossEntropyLoss().to(device),
    globaliter = 0
)
config2 = Config(
    trainloader = g_train_loader,
    testloader = g_valid_loader,
    model = resnet50_gender,
    device = device,
    optimizer = torch.optim.Adam(resnet50_gender.parameters(), lr=lr),
    criterion= nn.CrossEntropyLoss().to(device),
    globaliter = 0
)
config3 = Config(
    trainloader = a_train_loader,
    testloader = a_valid_loader,
    model = resnet50,
    device = device,
    optimizer = torch.optim.Adam(resnet50.parameters(), lr=lr),
    criterion= nn.CrossEntropyLoss().to(device),
    globaliter = 0
)

In [53]:
class train_test():
      def __init__(self, config,mode=None):
        # 파라미터 인자
        self.trainloader = config.trainloader
        self.testloader = config.testloader
        self.model = config.model
        self.device = config.device
        self.optimizer = config.optimizer
        self.criterion = config.criterion
        self.globaliter = config.globaliter
        self.min_loss = float("inf")
        self.mode = mode
      
      def train(self, epochs, log_interval, lr_sche):
          self.model.train()
          for epoch in range(1, epochs + 1 ):  # epochs 루프
              running_loss = 0.0
              lr_sche.step()
              for i, data in enumerate(self.trainloader, 0): # batch 루프
                  self.globaliter += 1
                  inputs, labels = data # input data, label 분리
                  inputs = inputs.to(self.device)
                  labels = labels.to(self.device)

                  self.optimizer.zero_grad() 

                  # forward + backward + optimize
                  outputs = self.model(inputs)
                  loss = self.criterion(outputs, labels)
                  loss.backward()
                  self.optimizer.step()
                  running_loss += loss.item()

                  # 30 iteration마다 acc & loss 출력
                  if i % log_interval == log_interval -1 : # i는 1에포크의 iteration
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tlearningLoss: {:.6f}\twhole_loss: {:.6f} '.format(
                        epoch, i*len(inputs), len(self.trainloader.dataset),
                        100. * i*len(inputs) / len(self.trainloader.dataset), 
                        running_loss / log_interval,
                        loss.item()))
                    running_loss = 0.0

              with torch.no_grad():
                  self.model.eval()
                  correct = 0
                  total = 0
                  test_loss = 0
                  labels_list = []
                  predict_list = []
                  acc = []
                  for k, data in enumerate(self.testloader, 0):
                    images, labels = data
                   
                    images = images.to(self.device)
                    labels = labels.to(self.device)
                    outputs = self.model(images)

                    labels_list.append(labels.detach().cpu().numpy())
                    _, predicted = torch.max(outputs.data, 1)
                    predict_list.append(predicted.detach().cpu().numpy())
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    test_loss += self.criterion(outputs, labels).item()
                    acc.append(100 * correct/total)
                  
                  labels_list = np.concatenate(labels_list)
                  predict_list = np.concatenate(predict_list)
                  score = f1_score(labels_list, predict_list, average='macro')

                  if test_loss < self.min_loss:
                    self.min_loss = test_loss
                    print(self.mode, "Minimum ::",self.min_loss)
                    torch.save({
                        'model': self.model.state_dict(),
                        'optimizer': self.optimizer.state_dict()
                    }, PATH + f'{self.mode}_all.tar')
                    

                  print('\nTest set : Average loss:{:.4f}, Accuracy: {}/{}({:.0f}%), F1-score: {}\n'.format(
                      test_loss, correct, total, 100 * correct/total, score
                  ))

In [54]:
mask_model = train_test(config1, mode="mask")
gender_model= train_test(config2, mode="gender")
age_model = train_test(config3, mode="ageCate")

In [55]:
m_lr_sche = optim.lr_scheduler.StepLR(config1.optimizer, step_size=10000, gamma=0.5) # 20 step마다 lr조정
g_lr_sche = optim.lr_scheduler.StepLR(config2.optimizer, step_size=10000, gamma=0.5) # 20 step마다 lr조정
a_lr_sche = optim.lr_scheduler.StepLR(config3.optimizer, step_size=10000, gamma=0.5) # 20 step마다 lr조정
log_interval = 175


In [56]:
mask_model.train(epochs, log_interval,m_lr_sche)

Train Epoch: 1 [2784/15120 (18%)]	learningLoss: 0.014515	whole_loss: 0.001749 
Train Epoch: 1 [5584/15120 (37%)]	learningLoss: 0.017357	whole_loss: 0.002388 
Train Epoch: 1 [8384/15120 (55%)]	learningLoss: 0.007657	whole_loss: 0.006423 
Train Epoch: 1 [11184/15120 (74%)]	learningLoss: 0.008475	whole_loss: 0.002604 
Train Epoch: 1 [13984/15120 (92%)]	learningLoss: 0.016981	whole_loss: 0.001159 
mask Minimum :: 8.761815305224104

Test set : Average loss:8.7618, Accuracy: 3744/3780(99%), F1-score: 0.9858647811639601

Train Epoch: 2 [2784/15120 (18%)]	learningLoss: 0.004515	whole_loss: 0.000001 
Train Epoch: 2 [5584/15120 (37%)]	learningLoss: 0.008301	whole_loss: 0.000450 
Train Epoch: 2 [8384/15120 (55%)]	learningLoss: 0.021842	whole_loss: 0.104162 
Train Epoch: 2 [11184/15120 (74%)]	learningLoss: 0.016143	whole_loss: 0.000009 
Train Epoch: 2 [13984/15120 (92%)]	learningLoss: 0.010561	whole_loss: 0.000009 
mask Minimum :: 4.572922144206842

Test set : Average loss:4.5729, Accuracy: 3765/3

In [60]:
gender_model.train(epochs, log_interval,g_lr_sche)

Train Epoch: 1 [2784/15120 (18%)]	learningLoss: 0.034413	whole_loss: 0.005132 
Train Epoch: 1 [5584/15120 (37%)]	learningLoss: 0.037141	whole_loss: 0.008287 
Train Epoch: 1 [8384/15120 (55%)]	learningLoss: 0.029943	whole_loss: 0.256830 
Train Epoch: 1 [11184/15120 (74%)]	learningLoss: 0.028811	whole_loss: 0.010416 
Train Epoch: 1 [13984/15120 (92%)]	learningLoss: 0.059230	whole_loss: 0.021733 
gender Minimum :: 37.93166743044276

Test set : Average loss:37.9317, Accuracy: 3620/3780(96%), F1-score: 0.9558985538400779

Train Epoch: 2 [2784/15120 (18%)]	learningLoss: 0.065627	whole_loss: 0.011559 
Train Epoch: 2 [5584/15120 (37%)]	learningLoss: 0.065409	whole_loss: 0.043910 
Train Epoch: 2 [8384/15120 (55%)]	learningLoss: 0.041438	whole_loss: 0.157047 
Train Epoch: 2 [11184/15120 (74%)]	learningLoss: 0.055116	whole_loss: 0.001200 
Train Epoch: 2 [13984/15120 (92%)]	learningLoss: 0.046219	whole_loss: 0.124955 

Test set : Average loss:42.5523, Accuracy: 3595/3780(95%), F1-score: 0.94942635

In [61]:
age_model.train(epochs, log_interval,a_lr_sche)

Train Epoch: 1 [2784/15120 (18%)]	learningLoss: 1.597810	whole_loss: 0.358551 
Train Epoch: 1 [5584/15120 (37%)]	learningLoss: 0.500214	whole_loss: 0.411356 
Train Epoch: 1 [8384/15120 (55%)]	learningLoss: 0.461373	whole_loss: 0.577858 
Train Epoch: 1 [11184/15120 (74%)]	learningLoss: 0.427987	whole_loss: 0.423450 
Train Epoch: 1 [13984/15120 (92%)]	learningLoss: 0.376770	whole_loss: 0.197144 
ageCate Minimum :: 78.19459466263652

Test set : Average loss:78.1946, Accuracy: 3304/3780(87%), F1-score: 0.7082520819145356

Train Epoch: 2 [2784/15120 (18%)]	learningLoss: 0.365023	whole_loss: 0.113317 
Train Epoch: 2 [5584/15120 (37%)]	learningLoss: 0.328279	whole_loss: 0.135725 
Train Epoch: 2 [8384/15120 (55%)]	learningLoss: 0.323985	whole_loss: 0.475490 
Train Epoch: 2 [11184/15120 (74%)]	learningLoss: 0.273864	whole_loss: 0.424163 
Train Epoch: 2 [13984/15120 (92%)]	learningLoss: 0.304876	whole_loss: 0.254563 
ageCate Minimum :: 75.78346095979214

Test set : Average loss:75.7835, Accuracy

In [72]:
PATH = './weights/'
m_checkpoint = torch.load(PATH + 'mask_all.tar') 

mask_model.model.load_state_dict(m_checkpoint['model'])
mask_model.optimizer.load_state_dict(m_checkpoint['optimizer'])

g_checkpoint = torch.load(PATH + 'gender_all.tar') 

gender_model.model.load_state_dict(g_checkpoint['model'])
gender_model.optimizer.load_state_dict(g_checkpoint['optimizer'])

a_checkpoint = torch.load(PATH + 'ageCate_all.tar') 

age_model.model.load_state_dict(a_checkpoint['model'])
age_model.optimizer.load_state_dict(a_checkpoint['optimizer'])


In [73]:
with torch.no_grad():
    mask_model.model.eval()
    gender_model.model.eval()
    age_model.model.eval()
    
    sub_list = []
    mask_list = []
    gender_list = []
    age_list = []

    for eval_x in eval_loader:
        outputs1 = mask_model.model(eval_x.cuda())
        outputs2 = gender_model.model(eval_x.cuda())
        outputs3 = age_model.model(eval_x.cuda())

        _, predicted1 = torch.max(outputs1.data, 1)
        _, predicted2 = torch.max(outputs2.data, 1)
        _, predicted3 = torch.max(outputs3.data, 1)

        sub_list.append([int(predicted1),int(predicted2),int(predicted3)])

In [74]:
len(sub_list)

12600

In [ ]:
classes_dict = {
        'Wear' : 0, 'NotWear' : 1, 'Incorrect' : 2,
        "female" : 1, "male" :0,
        'upper60' : 2 , "30to60":1, "lower30" :0,
        
    }

In [75]:
str(sub_list[0])

'[0, 0, 0]'

In [76]:
answer_dict = {
    '[0, 0, 0]' : 0, '[0, 0, 1]' : 1,
    '[0, 0, 2]' : 2, '[0, 1, 0]':3, '[0, 1, 1]':4, '[0, 1, 2]':5,
    '[2, 0, 0]':6, '[2, 0, 1]' :7, '[2, 0, 2]':8, '[2, 1, 0]':9, '[2, 1, 1]':10,
    '[2, 1, 2]':11, '[1, 0, 0]':12, '[1, 0, 1]':13, '[1, 0, 2]':14, '[1, 1, 0]':15, '[1, 1, 1]': 16,
    '[1, 1, 2]':17
}

In [77]:
submission = pd.read_csv('/opt/ml/input/data/eval/info.csv')

In [78]:
len(submission)

12600

In [79]:
len(sub_list)

12600

In [80]:
ans = [answer_dict[str(list_ans)] for list_ans in sub_list]

In [81]:
sub_dict = {key:val for key, val in zip(eval_set.get_id_list(), ans)}

In [82]:
final_ans = [sub_dict[id_img] for id_img in submission['ImageID'].values.tolist()]
submission['ans'] = final_ans

In [86]:
submission.to_csv('./submission.csv')

In [87]:
submission

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,13
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,0
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,16
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,3
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,13
